In [ ]:
import io 
from xml.etree import ElementTree
from lxml import etree
from pprint import pprint
import os
from os import listdir
from os.path import isfile, join 
import pandas as pd
import pdfminer 
import re

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #   
# Loop over pdf files and create a new .xml file for each pdf file. 

directory = '/scicomp/home/nwx6/jupyter/pdf_script_test/Unlocked_pdf_files_1'
# parse name to grab (pdf) --> grab spb number --> patient # 
patient_number = 1 #assign variable name 
print ("Calling PDFDUMP.py")

for filename in os.listdir(directory):
    #print(filename)
    if filename.endswith(".pdf"):
        
         #If file name has a string of 9 digits, save as a list, check for empty list and return the value.
        if re.findall("\d{9}", filename):
            gottcha_password=re.findall("\d{9}", filename) #returns a list, and we want to use the first entry in that list.
            #print(gottcha_password) 
        
        pathname = os.path.join(directory, filename)
        # >> will write everything to one .xml 
        data = os.system('dumppdf.py -a ' + pathname + ' > ' + '/scicomp/home/nwx6/jupyter/pdf_script_test/Output_xml_files/' + gottcha_password[0] + str(number) + '.xml') 
        patient_number = patient_number + 1
        
    else:
        #print("Invaild pdf file")  
        continue

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# initialize empty pd dataframe
initial_column=['VSPB_number']
final_df = pd.DataFrame(columns=initial_column)

# Loop over cleaned .xml files and parse.
print ("Opening XML output")

path = '/scicomp/home-pure/nwx6/jupyter/pdf_script_test/Output_xml_files/'
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
        
    if re.findall("\d{9}", filename):
        VSPB_number=re.findall("\d{9}", filename) #returns a list, and we want to use the first entry in that list.
        
    fullname = os.path.join(path, filename)
    parser = etree.XMLParser(recover = True) # Ignore bad characters in xml
    tree = ElementTree.parse(fullname, parser)
    lastnode = ""
    lastnode2 = ""
    list = {}
    entry = {}
    
    for node in tree.iter(): # Run through the tree..        
        # Check if New node
        if node.tag == "key" and node.text == "T":
            lastnode = node.tag + node.text
        elif lastnode == "keyT":
            for child in node.iter():
                entry["ID"] = child.text
            lastnode = ""

        if node.tag == "key" and node.text == "V":
            lastnode2 = node.tag + node.text
        elif lastnode2 == "keyV":
            for child in node.iter():
                if child.tag == "string":
                    if 'ID' in entry:
                        entry["Value"] = child.text
                        list[entry["ID"]] = entry["Value"]
                        entry = {}
            lastnode2 = ""
   

    #pprint(list)
    #This code takes the xml data and converts it into dictionary key:value pairs.
    #We then read this transposed data into a temporary data frame.
    #Then append these data frames together.
    
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
    #Note that we are still in the for loop and only working with a single *.xml file.
    test_df=pd.DataFrame.from_dict(list, orient = 'index')
    df1_transposed = test_df.transpose()
    df1_transposed['VSPB_number']=VSPB_number[0]
    #print(df1_transposed)
    final_df=final_df.append(df1_transposed)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #  

print(final_df)
final_df.to_excel('/scicomp/home-pure/nwx6/jupyter/pdf_script_test/Final_db/final_df_summary_V2.xlsx')